In [3]:
pip install git+https://github.com/huggingface/transformers 

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-sxn4bv3d
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-sxn4bv3d
  Resolved https://github.com/huggingface/transformers to commit 58e3d23e97078f361a533b9ec4a6a2de674ea52a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.38.0.dev0-py3-none-any.whl size=8458697 sha256=9cc4a76cbb01c54b090b4175277c147af7d470e4801a1ebcadac14f9289407f6
  Stored in directory: /tmp/pip-ephem-wheel-cache-1pzgop0f/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.0
    Uninstalling transformers-4.37.0:
      Successfully uninstalled transformers-4.37.0

[notice] A new release of pip is availa

In [2]:
!pip install -q accelerate==0.21.0 bitsandbytes==0.40.2 

In [3]:
pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 1.3 MB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import transformers
import torch
import os
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)


# Hugging face repo name
model = "NousResearch/llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto" # if you have GPU
)

2024-02-11 02:51:05.319858: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-11 02:51:05.319986: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-11 02:51:05.639367: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [5]:
import json
# Reading the train dataset back
#with open('/kaggle/input/cf-dataset/train_dataset_final.json', 'r') as train_file:
   # train_dataset_back = json.load(train_file)

# Reading the test dataset back
with open('/kaggle/input/cf-dataset/test_dataset_final.json', 'r') as test_file:
    test_dataset_back = json.load(test_file)

# Display the sizes of the two datasets
#print(f"Size of the train dataset: {len(train_dataset_back)}")
print(f"Size of the test dataset: {len(test_dataset_back)}")

Size of the test dataset: 599


In [6]:
from sentence_transformers import SentenceTransformer, util

def extract_answer(json_value):
    start_pattern = 'Answer: '
    end_pattern = '!'

    start_index = json_value.find(start_pattern)
    end_index = json_value.find(end_pattern, start_index + len(start_pattern))
    #print(start_index)
    #print(end_index)
    if start_index != -1 and end_index != -1:
        extracted_text = json_value[start_index + len(start_pattern):end_index]
        return f"{extracted_text.strip()}"

    return "Answer not found."

# Function to calculate F1 score
def compute_perf_score(golds, preds):
    tp = sum(1 for g, p in zip(golds, preds) if g == 1 and p == 1)
    fp = sum(1 for g, p in zip(golds, preds) if g == 0 and p == 1)
    fn = sum(1 for g, p in zip(golds, preds) if g == 1 and p == 0)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f1_score, precision, recall

def calculate_similarity(sentence1, sentence2, model_name='paraphrase-MiniLM-L6-v2'):
    # Load pre-trained BERT model for sentence embeddings
    model = SentenceTransformer(model_name)

    # Encode sentences to get their embeddings
    embedding1 = model.encode(sentence1, convert_to_tensor=True)
    embedding2 = model.encode(sentence2, convert_to_tensor=True)

    # Calculate cosine similarity between the two sentence embeddings
    similarity_score = util.pytorch_cos_sim(embedding1, embedding2)[0][0].item()

    return similarity_score

# Example usage
sentence1 = "One day he won a hundred dollars."
sentence2 = "he won a hundred dollars."

similarity = calculate_similarity(sentence1, sentence2)
print(f"Similarity Score: {similarity}")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Similarity Score: 0.916245698928833


In [ ]:
for dictionary in test_dataset_back[0:5]:
    #print(dictionary)
    premise = dictionary['premise']
    initial = dictionary['initial']
    counterfactual = dictionary['counterfactual']
    label=dictionary['label']
    query = f"{dictionary['premise']} {dictionary['initial']} or {dictionary['counterfactual']}? Start answering the best of the two options using the label 'Answer:'. End the response with an '!'"
    response = pipeline(query, do_sample=True, top_k=10, top_p=0.9, temperature=0.2, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id, max_length=200)
    generated_text = response[0]['generated_text']

    print(generated_text)
    print('-'*20)
    print(extract_answer(generated_text))
    print('^'*40)

In [9]:
# Generate predictions and collect gold labels
def evaluate_perf(data):
    preds, golds = [], []
    correct_count = 0
    total_items = len(data)
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    skipped_items = 0
    #dictionary in test_dataset_back[0:10]
    for index, item in enumerate(data):
        premise = item['premise']
        initial = item['initial']
        counterfactual = item['counterfactual']
        label=item['label']
        query = f"I have a situation: premise:{premise}. What do you think is the most plausible reason: Initial: {initial} OR Counterfactual: {counterfactual}? Provide response in one sentence. Start answering using the label 'Answer:'.End the response with an '!'"
        response = pipeline(query, do_sample=True, top_k=10, top_p=0.9, temperature=0.2, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id, truncation=True,max_length=150)
        generated_text = response[0]['generated_text']
        #print('*****',generated_text,'*****')
        correct_hypothesis = item['initial'] if item['label'] == 0 else item['counterfactual']
        print(f"\nProcessing item {index + 1}/{total_items}...")
        #print("Correct label:",correct_hypothesis)
        print('Label',item['label'])
        #print("Gen text:",extract_answer(generated_text))
    
    
        #print(generated_text)
        # Interpret the model's response
        similarity_score = calculate_similarity(correct_hypothesis, extract_answer(generated_text))
        print(similarity_score)
    
        if similarity_score>0.8:
            correct_count += 1
            if item['label'] == 1:
                true_positive += 1
                print('tp')
            else:
                true_negative += 1
                print('TN')
        else:
            if item['label'] == 1:
                false_negative += 1
                print('fn')
            else:
                false_positive += 1
                print('fp')
    
    print(f"TP: {true_positive:.2f}")
    print(f"FP: {false_positive:.2f}")
    print(f"FN: {false_negative:.2f}")
    print(f"total_items: {total_items}")
    print(f"CC: {correct_count}")
    #total_items -= skipped_items
    accuracy = correct_count / total_items if total_items > 0 else 0
    precision = true_positive / (true_positive + false_positive) if true_positive + false_positive > 0 else 0
    recall = true_positive / (true_positive + false_negative) if true_positive + false_negative > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
    print('-'*50)
    
    print(f"\nProcessed Items:{total_items}, Skipped Items: {skipped_items}")
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F1 Score: {f1_score * 100:.2f}%")

    return accuracy, precision, recall, f1_score

    return correct_count / len(data)

    #predicted_label = 0 if similarity_score<0.9 else 1
    #predicted_label = 0 if initial in extract_answer(generated_text) else 1
    #print(predicted_label)

    #preds.append(predicted_label)
    #golds.append(dictionary['label'])

# Calculate F1 score and accuracy
#f1_score, precision, recall = compute_perf_score(golds, preds)
#accuracy = sum(1 for g, p in zip(golds, preds) if g == p) / len(golds)

# Print the results
#print(f"F1 Score: {f1_score:.2f}")
#print(f"Accuracy: {accuracy:.2f}")
#print(f"Precision: {precision:.2f}")
#print(f"Recall: {recall:.2f}")

In [10]:
# Evaluate accuracy
accuracy, precision, recall, f1_score = evaluate_perf(test_dataset_back)


Processing item 1/599...
Label 1


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.14074310660362244
fn

Processing item 2/599...
Label 1


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.9999999403953552
TN

Processing item 599/599...
Label 0


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.9855916500091553
TN
TP: 4.00
FP: 390.00
FN: 64.00
total_items: 599
CC: 145
--------------------------------------------------

Processed Items:599, Skipped Items: 0
Accuracy: 24.21%
Precision: 1.02%
Recall: 5.88%
F1 Score: 1.73%
